## Preparing annotations

In [13]:
def get_speaker(d:dict):
    speaker = ""
    max_speaker = 0
    speaker_count = {}
    for item in d:
        speaker_key = item["speaker"]
        if not speaker_key=="Llama":
            if speaker_key in speaker_count:
                speaker_count[speaker_key] += 1
            else:
                speaker_count[speaker_key] = 1
    for s, count in speaker_count.items():
        if count > max_speaker:
            speaker = s
            max_speaker = count
    return speaker

In [75]:
def get_annotator(d:dict):
    for item in d:
        if "Annotator" in item:
            annotator = item["Annotator"]
            if not annotator=="auto":
                return annotator
    return ""

In [90]:
import json
import glob
import zipfile
#import pandas as pd

speaker_dict = {}
chat_folder = "/Users/piek/Desktop/t-MA-HLT-introduction-2024/lab-assignments/HLT-2024-lab0-submissions/combined"
all_conversations = []
for filename in glob.glob(chat_folder + '/*.json'):
    with open(filename) as f:
       # print(filename)
        d = json.load(f)
        speaker_key = get_speaker(d)
        if speaker_key in speaker_dict:
            speaker_dict[speaker_key].append(d)
        else:
            speaker_dict[speaker_key] = [d]

for speaker in speaker_dict:
    print(speaker, len(speaker_dict[speaker]))

Hello 1
Wilbur 2
Leon 2
Hello! My name is Alice. 1
Coins 2
Robert 1
Sissy 2
Coco 1
Moon. 1
Igor 1
Pretzel. 2
Kristof 1
Charles 1
Boris 1
Olaf 1
Carol 2
Bassie 2
Sarah 1
Big Head 1
Hey, I'm Lolamama. 2
Jake 2
Dorkus 1
Artemidorus 1
Hajmola 1
Bob 1
Hi, my name is Anna. 1
Amber 1


## Remove unannotated turns when doubled

In [91]:
cleaned_dict = {}
for speaker in speaker_dict:
    if len(speaker_dict[speaker])==1:
        cleaned_dict[speaker] = speaker_dict[speaker][0]
    else:
        if get_annotator(speaker_dict[speaker][0]):
            cleaned_dict[speaker] = speaker_dict[speaker][0]
        elif get_annotator(speaker_dict[speaker][1]):
            cleaned_dict[speaker] = speaker_dict[speaker][1]
        else:
            print(speaker, "No annotations in either, choosing the first")
            cleaned_dict[speaker] = speaker_dict[speaker][0]
        print(speaker, "annotator is", get_annotator(cleaned_dict[speaker]))

for speaker in cleaned_dict:
    data = cleaned_dict[speaker]
    annotator = get_annotator(data)
    if not annotator:
        for item in data:
            speaker_turn = item['speaker']
            if speaker_turn == "Llama":
                item["Annotator"] = "auto"
                item["Gold"]= "neutral"
            else:
                item["Annotator"] = ""
                item["Gold"]= ""
    print(speaker, len(cleaned_dict[speaker]))

Wilbur annotator is Wilbur
Leon annotator is Manya
Coins annotator is Yuleum
Sissy annotator is Melina Paxinou
Pretzel. No annotations in either, choosing the first
Pretzel. annotator is 
Carol annotator is Martha
Bassie annotator is Sanne
Hey, I'm Lolamama. annotator is Yunchong Huang
Jake annotator is Elisabetta
Hello 40
Wilbur 44
Leon 52
Hello! My name is Alice. 42
Coins 22
Robert 56
Sissy 42
Coco 42
Moon. 42
Igor 42
Pretzel. 46
Kristof 28
Charles 34
Boris 42
Olaf 40
Carol 38
Bassie 28
Sarah 42
Big Head 44
Hey, I'm Lolamama. 52
Jake 40
Dorkus 40
Artemidorus 26
Hajmola 44
Bob 44
Hi, my name is Anna. 52
Amber 42


## Map spakers to friends

In [78]:
speaker_friends = [{"speaker": "Bassie", "friends": ["Sarah", "Pretzel."]},
{"speaker": "Sarah", "friends": ["Pretzel.", "Coco"]},
{"speaker": "Pretzel.", "friends": ["Coco", "Amber"]},
{"speaker": "Coco", "friends": ["Amber", "Jake"]},
{"speaker": "Amber", "friends": ["Jake", "Hajmola"]},
{"speaker": "Jake", "friends": ["Hajmola", "Moon."]},
{"speaker": "Hajmola", "friends": ["Moon.", "Hello"]},
{"speaker": "Moon.", "friends": ["Hello", "Hey, I'm Lolamama."]},
{"speaker": "Hello", "friends": ["Hey, I'm Lolamama.", "Hello! My name is Alice."]},
{"speaker": "Hey, I'm Lolamama.", "friends": ["Hello! My name is Alice.", "Artemidorus"]},
{"speaker": "Hello! My name is Alice.", "friends": ["Artemidorus", "Bob"]},
{"speaker": "Artemidorus", "friends": ["Bob", "Carol"]},
{"speaker": "Bob", "friends": ["Carol", "Charles"]},
{"speaker": "Carol", "friends": ["Charles", "Big Head"]},
{"speaker": "Charles", "friends": ["Big Head", "Wilbur"]},
{"speaker": "Big Head", "friends": ["Wilbur", "Dorkus"]},
{"speaker": "Wilbur", "friends": ["Dorkus", "Robert"]},
{"speaker": "Dorkus", "friends": ["Robert", "Hi, my name is Anna."]},
{"speaker": "Robert", "friends": ["Hi, my name is Anna.", "Coins"]},
{"speaker": "Hi, my name is Anna.", "friends": ["Coins", "Sissy"]},
{"speaker": "Coins", "friends": ["Sissy", "Boris"]},
{"speaker": "Sissy", "friends": ["Boris", "Olaf"]},
{"speaker": "Boris", "friends": ["Olaf", "Leon"]},
{"speaker": "Olaf", "friends": ["Leon", "Kristof"]},
{"speaker": "Leon", "friends": ["Kristof", "Igor"]},
{"speaker": "Kristof", "friends": ["Igor", "Bassie"]},
{"speaker": "Igor", "friends": ["Bassie", "Sarah"]}]

print(speaker_friends)

[{'speaker': 'Bassie', 'friends': ['Sarah', 'Pretzel.']}, {'speaker': 'Sarah', 'friends': ['Pretzel.', 'Coco']}, {'speaker': 'Pretzel.', 'friends': ['Coco', 'Amber']}, {'speaker': 'Coco', 'friends': ['Amber', 'Jake']}, {'speaker': 'Amber', 'friends': ['Jake', 'Hajmola']}, {'speaker': 'Jake', 'friends': ['Hajmola', 'Moon.']}, {'speaker': 'Hajmola', 'friends': ['Moon.', 'Hello']}, {'speaker': 'Moon.', 'friends': ['Hello', "Hey, I'm Lolamama."]}, {'speaker': 'Hello', 'friends': ["Hey, I'm Lolamama.", 'Hello! My name is Alice.']}, {'speaker': "Hey, I'm Lolamama.", 'friends': ['Hello! My name is Alice.', 'Artemidorus']}, {'speaker': 'Hello! My name is Alice.', 'friends': ['Artemidorus', 'Bob']}, {'speaker': 'Artemidorus', 'friends': ['Bob', 'Carol']}, {'speaker': 'Bob', 'friends': ['Carol', 'Charles']}, {'speaker': 'Carol', 'friends': ['Charles', 'Big Head']}, {'speaker': 'Charles', 'friends': ['Big Head', 'Wilbur']}, {'speaker': 'Big Head', 'friends': ['Wilbur', 'Dorkus']}, {'speaker': 'Wi

In [79]:
### Speaker check
for item in speaker_friends:
    speaker = item['speaker']
    friends = item['friends']
    if not speaker in cleaned_dict:
        print('Cannot find speaker', speaker)
        print('Aborting')
        break
    for f in friends:
        if not f in cleaned_dict:
            print('Cannot find friend', f)
            print('Aborting')
            break

In [92]:
import pathlib, os
current_working_directory = pathlib.Path.cwd()
folder = "annotation_tasks"
for item in speaker_friends:
    speaker = item['speaker']
    friends = item['friends']
    annotator = get_annotator(cleaned_dict[speaker])
    file = os.path.join(current_working_directory, folder, speaker+'.json')
    print(file)
    data = [cleaned_dict[speaker]]
    for f in friends:
        friend_data = cleaned_dict[f]
        new_friend_data = []
        for friend_item in friend_data:
            speaker_turn = friend_item['speaker']
            new_data = friend_item.copy()
            if not speaker_turn=='Llama':
                new_data["Annotator"]= annotator
                new_data["Gold"]=""
            new_friend_data.append(new_data)
        data.append(new_friend_data)
    with open(file,'w') as file:
        json.dump(data, file, indent = 4)

/Users/piek/Desktop/t-MA-HLT-introduction-2024/ma-hlt-labs-class/lab0.llama/annotation_tasks/Bassie.json
/Users/piek/Desktop/t-MA-HLT-introduction-2024/ma-hlt-labs-class/lab0.llama/annotation_tasks/Sarah.json
/Users/piek/Desktop/t-MA-HLT-introduction-2024/ma-hlt-labs-class/lab0.llama/annotation_tasks/Pretzel..json
/Users/piek/Desktop/t-MA-HLT-introduction-2024/ma-hlt-labs-class/lab0.llama/annotation_tasks/Coco.json
/Users/piek/Desktop/t-MA-HLT-introduction-2024/ma-hlt-labs-class/lab0.llama/annotation_tasks/Amber.json
/Users/piek/Desktop/t-MA-HLT-introduction-2024/ma-hlt-labs-class/lab0.llama/annotation_tasks/Jake.json
/Users/piek/Desktop/t-MA-HLT-introduction-2024/ma-hlt-labs-class/lab0.llama/annotation_tasks/Hajmola.json
/Users/piek/Desktop/t-MA-HLT-introduction-2024/ma-hlt-labs-class/lab0.llama/annotation_tasks/Moon..json
/Users/piek/Desktop/t-MA-HLT-introduction-2024/ma-hlt-labs-class/lab0.llama/annotation_tasks/Hello.json
/Users/piek/Desktop/t-MA-HLT-introduction-2024/ma-hlt-labs-c